In [1]:
# Cell 1: Imports and Data Preparation
import os
import pandas as pd
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import cv2
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

def prepare_kaggle_data():
    """Подготавливает DataFrame с путями к данным Kaggle"""
    file_list = []
    
    kaggle_dirs = [
        '../input/liver-tumor-segmentation',
        '../input/liver-tumor-segmentation-part-2'
    ]
    
    for base_dir in kaggle_dirs:
        if os.path.exists(base_dir):
            print(f"Processing directory: {base_dir}")
            for dirname, _, filenames in os.walk(base_dir):
                for filename in filenames:
                    if filename.endswith('.nii'):
                        file_list.append((dirname, filename))
    
    df_files = pd.DataFrame(file_list, columns=['dirname', 'filename'])
    df_files = df_files.sort_values(by=['filename'], ascending=True)
    
    print(f"Total files found: {len(df_files)}")
    
    # Добавляем маски
    df_files["mask_dirname"] = ""
    df_files["mask_filename"] = ""
    
    matched_count = 0
    for i in range(131):
        ct = f"volume-{i}.nii"
        mask = f"segmentation-{i}.nii"
        
        ct_match = df_files['filename'] == ct
        if ct_match.any():
            df_files.loc[ct_match, 'mask_filename'] = mask
            df_files.loc[ct_match, 'mask_dirname'] = "../input/liver-tumor-segmentation/segmentations"
            matched_count += 1
    
    print(f"Matched {matched_count} volume-mask pairs")
    
    # Фильтруем и добавляем пути
    df_volumes_with_masks = df_files[df_files['mask_filename'] != ''].copy()
    df_volumes_with_masks = df_volumes_with_masks.reset_index(drop=True)
    
    df_volumes_with_masks['ct_path'] = df_volumes_with_masks['dirname'] + '/' + df_volumes_with_masks['filename']
    df_volumes_with_masks['mask_path'] = df_volumes_with_masks['mask_dirname'] + '/' + df_volumes_with_masks['mask_filename']
    
    print(f"Final dataset: {len(df_volumes_with_masks)} volumes with masks")
    return df_volumes_with_masks

# Запускаем подготовку данных
print("🔧 Preparing Kaggle data...")
df_kaggle = prepare_kaggle_data()
df_kaggle.head()

🔧 Preparing Kaggle data...
Processing directory: ../input/liver-tumor-segmentation
Processing directory: ../input/liver-tumor-segmentation-part-2
Total files found: 262
Matched 131 volume-mask pairs
Final dataset: 131 volumes with masks


,dirname,filename,mask_dirname,mask_filename,ct_path,mask_path
0,../input/liver-tumor-segmentation/volume_pt1,volume-0.nii,../input/liver-tumor-segmentation/segmentations,segmentation-0.nii,../input/liver-tumor-segmentation/volume_pt1/v...,../input/liver-tumor-segmentation/segmentation...
1,../input/liver-tumor-segmentation/volume_pt1,volume-1.nii,../input/liver-tumor-segmentation/segmentations,segmentation-1.nii,../input/liver-tumor-segmentation/volume_pt1/v...,../input/liver-tumor-segmentation/segmentation...
2,../input/liver-tumor-segmentation/volume_pt1,volume-10.nii,../input/liver-tumor-segmentation/segmentations,segmentation-10.nii,../input/liver-tumor-segmentation/volume_pt1/v...,../input/liver-tumor-segmentation/segmentation...
3,../input/liver-tumor-segmentation-part-2/volum...,volume-100.nii,../input/liver-tumor-segmentation/segmentations,segmentation-100.nii,../input/liver-tumor-segmentation-part-2/volum...,../input/liver-tumor-segmentation/segmentation...
4,../input/liver-tumor-segmentation-part-2/volum...,volume-101.nii,../input/liver-tumor-segmentation/segmentations,segmentation-101.nii,../input/liver-tumor-segmentation-part-2/volum...,../input/liver-tumor-segmentation/segmentation...


In [4]:
# Cell 2: Preprocessing and Saving to Disk
def read_nii(filepath):
    """Читает .nii файл и возвращает numpy array"""
    ct_scan = nib.load(filepath)
    array = ct_scan.get_fdata()
    array = np.rot90(np.array(array))
    return array

def preprocess_and_save_data(df_files, output_dir='processed_data_256', image_size=256):
    """
    Предобрабатывает данные и сохраняет на диск
    Возвращает DataFrame с путями к обработанным файлам
    """
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(f'{output_dir}/images', exist_ok=True)
    os.makedirs(f'{output_dir}/masks', exist_ok=True)
    
    image_paths = []
    mask_paths = []
    volume_ids = []
    slice_ids = []
    
    print(f"🚀 Starting preprocessing - saving to {output_dir}")
    print(f"📐 Target size: {image_size}x{image_size}")
    
    for idx in tqdm(range(len(df_files)), desc="Processing volumes"):
        try:
            # Проверяем существование файлов
            ct_path = df_files.loc[idx, 'ct_path']
            mask_path = df_files.loc[idx, 'mask_path']
            
            if not os.path.exists(ct_path) or not os.path.exists(mask_path):
                print(f"⚠️ Files not found for volume {idx}")
                continue
            
            # Чтение данных
            ct_scan = read_nii(ct_path)
            mask = read_nii(mask_path)
            
            # Получаем ID volume из имени файла
            file_name = df_files.loc[idx, 'filename']
            volume_id = file_name.replace('volume-', '').replace('.nii', '')
            
            # Обрабатываем ВСЕ срезы
            num_slices = ct_scan.shape[2]
            
            for slice_idx in range(num_slices):
                # Извлекаем срез
                ct_slice = ct_scan[..., slice_idx]
                mask_slice = mask[..., slice_idx]
                
                # Нормализация CT среза [0, 1]
                ct_slice_normalized = (ct_slice - ct_slice.min()) / (ct_slice.max() - ct_slice.min() + 1e-8)
                
                # Конвертируем в uint8 [0, 255] для сохранения
                ct_slice_uint8 = (ct_slice_normalized * 255).astype(np.uint8)
                mask_slice_uint8 = mask_slice.astype(np.uint8)
                
                # Изменение размера
                ct_slice_resized = cv2.resize(ct_slice_uint8, (image_size, image_size))
                mask_slice_resized = cv2.resize(mask_slice_uint8, (image_size, image_size), 
                                               interpolation=cv2.INTER_NEAREST)
                
                # Сохранение изображений
                image_filename = f'volume_{volume_id}_slice_{slice_idx:03d}.png'
                mask_filename = f'volume_{volume_id}_slice_{slice_idx:03d}_mask.png'
                
                image_path = f'{output_dir}/images/{image_filename}'
                mask_path = f'{output_dir}/masks/{mask_filename}'
                
                # Сохраняем как PNG
                cv2.imwrite(image_path, ct_slice_resized)
                cv2.imwrite(mask_path, mask_slice_resized)
                
                # Сохраняем пути
                image_paths.append(image_path)
                mask_paths.append(mask_path)
                volume_ids.append(int(volume_id))
                slice_ids.append(slice_idx)
                
        except Exception as e:
            print(f"❌ Error processing volume {idx}: {e}")
            continue
    
    # Создаем DataFrame с путями
    df_processed = pd.DataFrame({
        'image_path': image_paths,
        'mask_path': mask_paths,
        'volume_id': volume_ids,
        'slice_id': slice_ids
    })
    
    # Сохраняем CSV с путями
    csv_path = f'{output_dir}/dataset.csv'
    df_processed.to_csv(csv_path, index=False)
    
    print(f"✅ Preprocessing completed!")
    print(f"📁 Saved {len(df_processed)} image-mask pairs")
    print(f"💾 CSV saved: {csv_path}")
    
    return df_processed

# Запускаем предобработку (это займет время)
print("🔄 Starting data preprocessing...")
df_processed = preprocess_and_save_data(df_kaggle, image_size=256)
df_processed.head()

🔄 Starting data preprocessing...
🚀 Starting preprocessing - saving to processed_data_256
📐 Target size: 256x256


Processing volumes:   0%|          | 0/131 [00:00<?, ?it/s]

✅ Preprocessing completed!
📁 Saved 58638 image-mask pairs
💾 CSV saved: processed_data_256/dataset.csv


,image_path,mask_path,volume_id,slice_id
0,processed_data_256/images/volume_0_slice_000.png,processed_data_256/masks/volume_0_slice_000_ma...,0,0
1,processed_data_256/images/volume_0_slice_001.png,processed_data_256/masks/volume_0_slice_001_ma...,0,1
2,processed_data_256/images/volume_0_slice_002.png,processed_data_256/masks/volume_0_slice_002_ma...,0,2
3,processed_data_256/images/volume_0_slice_003.png,processed_data_256/masks/volume_0_slice_003_ma...,0,3
4,processed_data_256/images/volume_0_slice_004.png,processed_data_256/masks/volume_0_slice_004_ma...,0,4


In [2]:
# Cell 3: Load Preprocessed Data
def load_preprocessed_data(data_dir='processed_data_256'):
    """Загружает готовые предобработанные данные"""
    csv_path = f'{data_dir}/dataset.csv'
    
    if not os.path.exists(csv_path):
        print(f"❌ CSV file not found: {csv_path}")
        print("Please run Cell 2 first!")
        return None
    
    df_processed = pd.read_csv(csv_path)
    
    # Проверяем существование файлов
    existing_files = 0
    for idx, row in df_processed.iterrows():
        if os.path.exists(row['image_path']) and os.path.exists(row['mask_path']):
            existing_files += 1
    
    print(f"📊 Loaded dataset: {len(df_processed)} pairs")
    print(f"✅ Files that exist: {existing_files}/{len(df_processed)}")
    
    if existing_files == 0:
        print("❌ No files found! Please check paths.")
        return None
    
    return df_processed

# Быстрая загрузка готовых данных
print("📥 Loading preprocessed data...")
df_ready = load_preprocessed_data()

if df_ready is not None:
    print("✅ Data loaded successfully!")
    df_ready.head()

📥 Loading preprocessed data...
📊 Loaded dataset: 58638 pairs
✅ Files that exist: 58638/58638
✅ Data loaded successfully!


In [3]:
# Cell 4: Dataset for Fast Loading
class LiverDataset(Dataset):
    def __init__(self, df, image_size=256, transform=None):
        self.df = df
        self.image_size = image_size
        self.transform = transform
        self.image_paths = df['image_path'].values
        self.mask_paths = df['mask_path'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # Быстрая загрузка готовых PNG
        image_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]
        
        # Загрузка изображений
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        # Нормализация [0, 255] -> [0, 1]
        image = image.astype(np.float32) / 255.0
        
        # Преобразование в тензоры
        image_tensor = torch.FloatTensor(image).unsqueeze(0)  # [1, H, W]
        mask_tensor = torch.LongTensor(mask)                  # [H, W]
        
        return image_tensor, mask_tensor

def create_data_loaders(df, batch_size=8, test_size=0.2):
    """Создает DataLoader'ы для обучения"""
    # Разделяем на train/val
    train_df, val_df = train_test_split(df, test_size=test_size, random_state=42, shuffle=True)
    
    print(f"📊 Training samples: {len(train_df)}")
    print(f"📊 Validation samples: {len(val_df)}")
    
    # Создаем датасеты
    train_dataset = LiverDataset(train_df)
    val_dataset = LiverDataset(val_df)
    
    # Создаем DataLoader'ы
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    return train_loader, val_loader

# Быстро создаем DataLoader'ы (секунды)
if df_ready is not None:
    print("🔄 Creating data loaders...")
    train_loader, val_loader = create_data_loaders(df_ready, batch_size=8)
    print("✅ Data loaders ready!")

🔄 Creating data loaders...
📊 Training samples: 46910
📊 Validation samples: 11728
✅ Data loaders ready!


In [4]:
# Cell 4.1: Create Optimized Datasets
def create_optimized_datasets(df, image_size=256, max_samples=None):
    """Создает оптимизированные датасеты с возможностью ограничения выборки"""
    
    # Ограничиваем количество samples для быстрого прототипирования
    if max_samples and len(df) > max_samples:
        print(f"⚠️ Limiting dataset from {len(df)} to {max_samples} samples for faster prototyping")
        df = df.sample(max_samples, random_state=42).reset_index(drop=True)
    
    # Разделяем на train/val
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
    
    print(f"📊 Training samples: {len(train_df)}")
    print(f"📊 Validation samples: {len(val_df)}")
    
    # Создаем датасеты
    train_dataset = LiverDataset(train_df, image_size=image_size)
    val_dataset = LiverDataset(val_df, image_size=image_size)
    
    return train_dataset, val_dataset, train_df, val_df

# Создаем оптимизированные датасеты (ограничиваем для скорости)
print("🔄 Creating optimized datasets...")
train_dataset, val_dataset, train_df, val_df = create_optimized_datasets(
    df_ready, 
    max_samples=2000  # Ограничиваем для быстрого тестирования
)

print("✅ Datasets created successfully!")

🔄 Creating optimized datasets...
⚠️ Limiting dataset from 58638 to 2000 samples for faster prototyping
📊 Training samples: 1600
📊 Validation samples: 400
✅ Datasets created successfully!


In [6]:
# Cell 5: U-Net Model Definition
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels=1, n_classes=3):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        # Encoder
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(64, 128))
        self.down2 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(128, 256))
        self.down3 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(256, 512))
        self.down4 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(512, 1024))

        # Decoder
        self.up1 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(1024, 512, kernel_size=3, padding=1)
        )
        self.conv1 = DoubleConv(1024, 512)
        
        self.up2 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(512, 256, kernel_size=3, padding=1)
        )
        self.conv2 = DoubleConv(512, 256)
        
        self.up3 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(256, 128, kernel_size=3, padding=1)
        )
        self.conv3 = DoubleConv(256, 128)
        
        self.up4 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(128, 64, kernel_size=3, padding=1)
        )
        self.conv4 = DoubleConv(128, 64)

        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)

    def forward(self, x):
        # Encoder
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        # Decoder
        x = self.up1(x5)
        x = torch.cat([x, x4], dim=1)
        x = self.conv1(x)

        x = self.up2(x)
        x = torch.cat([x, x3], dim=1)
        x = self.conv2(x)

        x = self.up3(x)
        x = torch.cat([x, x2], dim=1)
        x = self.conv3(x)

        x = self.up4(x)
        x = torch.cat([x, x1], dim=1)
        x = self.conv4(x)

        logits = self.outc(x)
        return logits

print("🧠 U-Net model defined successfully!")

🧠 U-Net model defined successfully!


In [7]:
# Cell 6: Optimized Training
def train_model_fast(model, train_loader, val_loader, num_epochs=25, device='cuda'):
    """Оптимизированная функция обучения"""
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    model.to(device)
    train_losses, val_losses = [], []
    
    # Уменьшаем вычисление IoU на каждом шаге для скорости
    calculate_iou_every = 50  # Считаем IoU только каждые 50 батчей
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0.0
        running_iou = 0.0
        iou_count = 0
        
        # Используем tqdm с минимальными обновлениями
        batch_iterator = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', 
                             mininterval=10)  # Обновляем раз в 10 секунд
        
        for batch_idx, (images, masks) in enumerate(batch_iterator):
            images, masks = images.to(device), masks.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            # IoU считаем реже для скорости
            if batch_idx % calculate_iou_every == 0:
                with torch.no_grad():
                    preds = torch.argmax(outputs, dim=1)
                    iou = calculate_iou(preds, masks)
                    running_iou += iou
                    iou_count += 1
            
            # Минимальное обновление прогресса
            if batch_idx % 100 == 0:
                batch_iterator.set_postfix({
                    'Loss': f'{loss.item():.4f}',
                    'Avg Loss': f'{running_loss/(batch_idx+1):.4f}'
                })
        
        # Validation (только в конце эпохи)
        model.eval()
        val_loss = 0.0
        val_iou = 0.0
        
        with torch.no_grad():
            # Берем только подмножество валидации для скорости
            val_samples = min(100, len(val_loader))
            for i, (images, masks) in enumerate(val_loader):
                if i >= val_samples:
                    break
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                loss = criterion(outputs, masks)
                val_loss += loss.item()
                
                preds = torch.argmax(outputs, dim=1)
                iou = calculate_iou(preds, masks)
                val_iou += iou
        
        # Calculate averages
        epoch_train_loss = running_loss / len(train_loader)
        epoch_val_loss = val_loss / val_samples
        epoch_train_iou = running_iou / iou_count if iou_count > 0 else 0
        epoch_val_iou = val_iou / val_samples
        
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'  Train Loss: {epoch_train_loss:.4f}, Train IoU: {epoch_train_iou:.4f}')
        print(f'  Val Loss: {epoch_val_loss:.4f}, Val IoU: {epoch_val_iou:.4f}')
        
        # Save best model
        if epoch_val_loss == min(val_losses):
            torch.save(model.state_dict(), 'best_unet_liver.pth')
            print(f'  💾 Best model saved!')
    
    return train_losses, val_losses

# Альтернатива: быстрая версия без tqdm
def train_model_very_fast(model, train_loader, val_loader, num_epochs=25, device='cuda'):
    """Сверхбыстрая версия без прогресс-бара"""
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    model.to(device)
    train_losses, val_losses = [], []
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0.0
        
        print(f'Epoch {epoch+1}/{num_epochs} - Training...')
        
        for batch_idx, (images, masks) in enumerate(train_loader):
            images, masks = images.to(device), masks.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            # Выводим прогресс каждые 100 батчей
            if batch_idx % 100 == 0:
                avg_loss = running_loss / (batch_idx + 1)
                print(f'  Batch {batch_idx}/{len(train_loader)}, Loss: {avg_loss:.4f}')
        
        # Validation
        model.eval()
        val_loss = 0.0
        
        print(f'Epoch {epoch+1}/{num_epochs} - Validation...')
        
        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                loss = criterion(outputs, masks)
                val_loss += loss.item()
        
        epoch_train_loss = running_loss / len(train_loader)
        epoch_val_loss = val_loss / len(val_loader)
        
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        
        print(f'Epoch {epoch+1} Complete:')
        print(f'  Train Loss: {epoch_train_loss:.4f}')
        print(f'  Val Loss: {epoch_val_loss:.4f}')
        
        if epoch_val_loss == min(val_losses):
            torch.save(model.state_dict(), 'best_unet_liver.pth')
            print(f'  💾 Best model saved!')
    
    return train_losses, val_losses

In [7]:
# Cell 7: Optimized Training Setup (FIXED)
def setup_fast_training(train_dataset, val_dataset):
    """Настройка быстрого обучения"""
    
    # 1. Оптимальный размер батча
    BATCH_SIZE = 16  # Начинаем с малого, можно увеличить если хватит памяти
    
    # 2. Оптимальное количество workers
    NUM_WORKERS = 4  # 0 для Windows, 2-4 для Linux
    
    print(f"⚙️ Training setup:")
    print(f"  Batch size: {BATCH_SIZE}")
    print(f"  Workers: {NUM_WORKERS}")
    print(f"  Training samples: {len(train_dataset)}")
    print(f"  Validation samples: {len(val_dataset)}")
    
    # 3. Создаем оптимизированные DataLoader'ы
    train_loader_fast = DataLoader(
        train_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=True, 
        num_workers=NUM_WORKERS,
        pin_memory=True,  # Ускоряет transfer на GPU
        persistent_workers=True if NUM_WORKERS > 0 else False
    )
    
    val_loader_fast = DataLoader(
        val_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False, 
        num_workers=NUM_WORKERS,
        pin_memory=True,
        persistent_workers=True if NUM_WORKERS > 0 else False
    )
    
    # 4. Создаем модель
    model_fast = UNet(n_channels=1, n_classes=3)
    
    # 5. Mixed precision scaler
    scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
    
    print(f"📊 Batches per epoch: {len(train_loader_fast)}")
    
    # Оцениваем время
    if len(train_loader_fast) > 0:
        estimated_time = len(train_loader_fast) * 0.2  # ~0.2 сек на батч
        print(f"⏱️ Estimated time per epoch: {estimated_time/60:.1f} minutes")
    
    return model_fast, train_loader_fast, val_loader_fast, scaler

# СУПЕР-БЫСТРАЯ ФУНКЦИЯ ОБУЧЕНИЯ
def train_model_super_fast(model, train_loader, val_loader, num_epochs=10, device='cuda'):
    """Сверхбыстрая версия обучения"""
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    model.to(device)
    train_losses, val_losses = [], []
    
    print("🚀 Starting SUPER FAST training...")
    
    for epoch in range(num_epochs):
        # === TRAINING ===
        model.train()
        running_loss = 0.0
        start_time = time.time()
        
        print(f'\nEpoch {epoch+1}/{num_epochs} - Training:')
        
        for batch_idx, (images, masks) in enumerate(train_loader):
            images, masks = images.to(device), masks.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            # Быстрый прогресс-бар каждые 10% 
            if batch_idx % max(1, len(train_loader) // 10) == 0:
                progress = (batch_idx + 1) / len(train_loader) * 100
                avg_loss = running_loss / (batch_idx + 1)
                print(f'  Progress: {progress:.0f}% | Loss: {avg_loss:.4f}')
        
        # === VALIDATION ===
        model.eval()
        val_loss = 0.0
        
        print(f'Epoch {epoch+1}/{num_epochs} - Validation...')
        
        with torch.no_grad():
            # Быстрая валидация на 20% данных
            val_batches = min(20, len(val_loader))
            for i, (images, masks) in enumerate(val_loader):
                if i >= val_batches:
                    break
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                loss = criterion(outputs, masks)
                val_loss += loss.item()
        
        # Calculate metrics
        epoch_train_loss = running_loss / len(train_loader)
        epoch_val_loss = val_loss / val_batches
        epoch_time = time.time() - start_time
        
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        
        print(f'✅ Epoch {epoch+1} Complete:')
        print(f'   Time: {epoch_time/60:.1f} min')
        print(f'   Train Loss: {epoch_train_loss:.4f}')
        print(f'   Val Loss: {epoch_val_loss:.4f}')
        
        # Save best model
        if epoch_val_loss == min(val_losses):
            torch.save(model.state_dict(), 'best_unet_liver.pth')
            print(f'   💾 Best model saved!')
    
    return train_losses, val_losses

# ЗАПУСК ОПТИМИЗИРОВАННОГО ОБУЧЕНИЯ
import time

print("⚡ Setting up fast training...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model_fast, train_loader_fast, val_loader_fast, scaler = setup_fast_training(train_dataset, val_dataset)

# Запускаем сверхбыстрое обучение
train_losses, val_losses = train_model_super_fast(
    model_fast, train_loader_fast, val_loader_fast, 
    num_epochs=10,  # Начинаем с 5 эпох
    device=device
)

torch.save(model_fast.state_dict(), 'final_unet_liver_fast.pth')
print("🎉 FAST training completed!")

⚡ Setting up fast training...
Using device: cuda
⚙️ Training setup:
  Batch size: 16
  Workers: 4
  Training samples: 1600
  Validation samples: 400
📊 Batches per epoch: 100
⏱️ Estimated time per epoch: 0.3 minutes
🚀 Starting SUPER FAST training...

Epoch 1/10 - Training:
  Progress: 1% | Loss: 1.3105


KeyboardInterrupt: 

In [8]:
# Cell 4.2: Full Dataset Setup
def create_full_datasets(df, image_size=256):
    """Создает датасеты из ВСЕХ доступных данных"""
    
    print(f"📊 Original dataset size: {len(df)} samples")
    
    # Разделяем на train/val
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
    
    print(f"📊 Training samples: {len(train_df)}")
    print(f"📊 Validation samples: {len(val_df)}")
    print(f"📊 Total samples: {len(train_df) + len(val_df)}")
    
    # Создаем датасеты
    train_dataset_full = LiverDataset(train_df, image_size=image_size)
    val_dataset_full = LiverDataset(val_df, image_size=image_size)
    
    return train_dataset_full, val_dataset_full, train_df, val_df

# Создаем полный датасет
print("🔄 Creating FULL datasets...")
train_dataset_full, val_dataset_full, train_df_full, val_df_full = create_full_datasets(df_ready)

print("✅ Full datasets created!")
print(f"🎯 You now have {len(train_dataset_full)} training samples")
print(f"🎯 and {len(val_dataset_full)} validation samples")

🔄 Creating FULL datasets...
📊 Original dataset size: 58638 samples
📊 Training samples: 46910
📊 Validation samples: 11728
📊 Total samples: 58638
✅ Full datasets created!
🎯 You now have 46910 training samples
🎯 and 11728 validation samples


In [9]:
# Cell 7.2: Full Dataset Training
def setup_full_training(train_dataset, val_dataset):
    """Настройка обучения на полном датасете"""
    
    # Оптимальные настройки для полного датасета
    BATCH_SIZE = 8  # Уменьшаем batch size для полного датасета
    NUM_WORKERS = 4
    
    print(f"⚡ FULL DATASET Training setup:")
    print(f"  Batch size: {BATCH_SIZE}")
    print(f"  Workers: {NUM_WORKERS}")
    print(f"  Training samples: {len(train_dataset)}")
    print(f"  Validation samples: {len(val_dataset)}")
    
    # DataLoader'ы для полного датасета
    train_loader_full = DataLoader(
        train_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=True, 
        num_workers=NUM_WORKERS,
        pin_memory=True
    )
    
    val_loader_full = DataLoader(
        val_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False, 
        num_workers=NUM_WORKERS,
        pin_memory=True
    )
    
    model_full = UNet(n_channels=1, n_classes=3)
    
    print(f"📊 Batches per epoch: {len(train_loader_full)}")
    
    # Оценка времени для полного датасета
    estimated_time = len(train_loader_full) * 0.1  # ~0.1 сек на батч с GPU
    print(f"⏱️ Estimated time per epoch: {estimated_time/60:.1f} minutes")
    
    return model_full, train_loader_full, val_loader_full

# Функция обучения на полном датасете
def train_full_dataset(model, train_loader, val_loader, num_epochs=10, device='cuda'):
    """Обучение на полном датасете"""
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    model.to(device)
    train_losses, val_losses = [], []
    
    print(f"🚀 Starting FULL DATASET training on {len(train_loader.dataset)} samples...")
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0.0
        start_time = time.time()
        
        print(f'\n🎯 Epoch {epoch+1}/{num_epochs} - Full Dataset')
        print('-' * 50)
        
        for batch_idx, (images, masks) in enumerate(train_loader):
            images, masks = images.to(device), masks.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            # Прогресс каждые 5%
            if batch_idx % max(1, len(train_loader) // 20) == 0:
                progress = (batch_idx + 1) / len(train_loader) * 100
                avg_loss = running_loss / (batch_idx + 1)
                print(f'  Progress: {progress:.0f}% | Loss: {avg_loss:.4f}')
        
        # Validation
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                loss = criterion(outputs, masks)
                val_loss += loss.item()
        
        # Calculate metrics
        epoch_train_loss = running_loss / len(train_loader)
        epoch_val_loss = val_loss / len(val_loader)
        epoch_time = time.time() - start_time
        
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        
        print(f'✅ Epoch {epoch+1} Complete:')
        print(f'   ⏱️  Time: {epoch_time/60:.1f} min')
        print(f'   📉 Train Loss: {epoch_train_loss:.4f}')
        print(f'   📊 Val Loss: {epoch_val_loss:.4f}')
        
        # Save best model
        if epoch_val_loss == min(val_losses):
            torch.save(model.state_dict(), 'best_unet_liver_full.pth')
            print(f'   💾 Best model saved!')
    
    return train_losses, val_losses

# ЗАПУСК ОБУЧЕНИЯ НА ПОЛНОМ ДАТАСЕТЕ
print("⚡ Setting up FULL dataset training...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_full, train_loader_full, val_loader_full = setup_full_training(train_dataset_full, val_dataset_full)

⚡ Setting up FULL dataset training...
⚡ FULL DATASET Training setup:
  Batch size: 8
  Workers: 4
  Training samples: 46910
  Validation samples: 11728
📊 Batches per epoch: 5864
⏱️ Estimated time per epoch: 9.8 minutes


In [16]:


# Запускаем обучение на полном датасете
train_losses, val_losses = train_full_dataset(
    model_full, train_loader_full, val_loader_full, 
    num_epochs=5,  # Можно увеличить до 10-20
    device=device
)

torch.save(model_full.state_dict(), 'final_unet_liver_full.pth')
print("🎉 FULL DATASET training completed!")

⚡ Setting up FULL dataset training...
⚡ FULL DATASET Training setup:
  Batch size: 8
  Workers: 4
  Training samples: 46910
  Validation samples: 11728
📊 Batches per epoch: 5864
⏱️ Estimated time per epoch: 9.8 minutes
🚀 Starting FULL DATASET training on 46910 samples...

🎯 Epoch 1/5 - Full Dataset
--------------------------------------------------
  Progress: 0% | Loss: 1.0412
  Progress: 5% | Loss: 0.3912
  Progress: 10% | Loss: 0.2804
  Progress: 15% | Loss: 0.2175
  Progress: 20% | Loss: 0.1782
  Progress: 25% | Loss: 0.1518
  Progress: 30% | Loss: 0.1322
  Progress: 35% | Loss: 0.1178
  Progress: 40% | Loss: 0.1063
  Progress: 45% | Loss: 0.0972
  Progress: 50% | Loss: 0.0895
  Progress: 55% | Loss: 0.0832
  Progress: 60% | Loss: 0.0779
  Progress: 65% | Loss: 0.0734
  Progress: 70% | Loss: 0.0693
  Progress: 75% | Loss: 0.0656
  Progress: 80% | Loss: 0.0624
  Progress: 85% | Loss: 0.0595
  Progress: 90% | Loss: 0.0571
  Progress: 95% | Loss: 0.0548
  Progress: 100% | Loss: 0.0526

In [ ]:
# Cell 8: Model Testing and Evaluation with Memory Control
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import os
import pickle
import gc

def calculate_iou_per_class(preds, targets, n_classes=3):
    """Calculate IoU for each class separately"""
    ious = []
    
    for cls in range(n_classes):
        pred_cls = preds == cls
        target_cls = targets == cls
        
        intersection = (pred_cls & target_cls).float().sum((1, 2))
        union = (pred_cls | target_cls).float().sum((1, 2))
        
        iou = (intersection + 1e-6) / (union + 1e-6)
        ious.append(iou.mean().item())
    
    return ious

def calculate_dice_per_class(preds, targets, n_classes=3):
    """Calculate Dice coefficient for each class"""
    dice_scores = []
    
    for cls in range(n_classes):
        pred_cls = preds == cls
        target_cls = targets == cls
        
        intersection = (pred_cls & target_cls).float().sum((1, 2))
        union = pred_cls.float().sum((1, 2)) + target_cls.float().sum((1, 2))
        
        dice = (2. * intersection + 1e-6) / (union + 1e-6)
        dice_scores.append(dice.mean().item())
    
    return dice_scores

def test_model_incremental(model, test_loader, device='cuda', 
                          chunk_size=150, results_dir='incremental_results',
                          resume_from=None):
    """
    Test model incrementally with memory control
    
    Args:
        model: Trained model
        test_loader: DataLoader with test data
        device: Device to run on
        chunk_size: Number of samples to process at once
        results_dir: Directory to save results
        resume_from: Resume from specific chunk (None for fresh start)
    """
    model.eval()
    
    total_samples = len(test_loader)
    print(f"🧪 Starting incremental testing on {total_samples} samples...")
    print(f"📦 Chunk size: {chunk_size}")
    print(f"💾 Results directory: {results_dir}")
    
    # Create results directory
    os.makedirs(results_dir, exist_ok=True)
    
    # Initialize or load existing results
    if resume_from is not None and os.path.exists(f'{results_dir}/current_progress.pkl'):
        with open(f'{results_dir}/current_progress.pkl', 'rb') as f:
            progress = pickle.load(f)
        print(f"🔄 Resuming from chunk {progress['current_chunk']}, sample {progress['processed_samples']}")
    else:
        progress = {
            'current_chunk': 0,
            'processed_samples': 0,
            'all_preds': [],
            'all_targets': [],
            'chunk_metrics': [],
            'total_iou': 0.0,
            'total_dice': 0.0,
            'total_class_ious': [0.0, 0.0, 0.0],
            'total_class_dice': [0.0, 0.0, 0.0]
        }
        # Clean previous results
        for f in os.listdir(results_dir):
            if f.startswith('chunk_') and f.endswith('.pkl'):
                os.remove(f'{results_dir}/{f}')
    
    start_chunk = progress['current_chunk']
    start_sample = progress['processed_samples']
    
    # Process in chunks
    for chunk_idx in range(start_chunk, (total_samples + chunk_size - 1) // chunk_size):
        start_idx = chunk_idx * chunk_size
        end_idx = min((chunk_idx + 1) * chunk_size, total_samples)
        
        print(f"\n{'='*50}")
        print(f"🔄 Processing chunk {chunk_idx + 1}/{(total_samples + chunk_size - 1)//chunk_size}")
        print(f"📊 Samples {start_idx} to {end_idx - 1}")
        
        # Process current chunk
        chunk_results = process_chunk(
            model, test_loader, device, start_idx, end_idx, chunk_idx
        )
        
        # Update progress
        progress = update_progress(progress, chunk_results, results_dir)
        
        # Save chunk results
        save_chunk_results(chunk_results, chunk_idx, results_dir)
        
        # Clear memory
        clear_memory()
        
        # Print intermediate results
        print_intermediate_results(progress, chunk_idx + 1)
        
        # Save checkpoint after each chunk
        save_checkpoint(progress, results_dir)
    
    # Final results
    final_results = compile_final_results(progress, results_dir)
    
    return final_results

def process_chunk(model, test_loader, device, start_idx, end_idx, chunk_idx):
    """Process a single chunk of data"""
    chunk_iou = 0.0
    chunk_dice = 0.0
    chunk_class_ious = [0.0, 0.0, 0.0]
    chunk_class_dice = [0.0, 0.0, 0.0]
    
    chunk_preds = []
    chunk_targets = []
    chunk_samples = 0
    
    print(f"  Processing samples {start_idx} to {end_idx-1}...")
    
    with torch.no_grad():
        for i, (images, masks) in enumerate(test_loader):
            if i < start_idx:
                continue
            if i >= end_idx:
                break
                
            # Move data to GPU
            images = images.to(device, non_blocking=True)
            masks = masks.to(device, non_blocking=True)
            
            # Forward pass
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            
            # Calculate metrics
            ious = calculate_iou_per_class(preds, masks)
            dice_scores = calculate_dice_per_class(preds, masks)
            
            chunk_iou += np.mean(ious)
            chunk_dice += np.mean(dice_scores)
            
            for cls in range(3):
                chunk_class_ious[cls] += ious[cls]
                chunk_class_dice[cls] += dice_scores[cls]
            
            # Store predictions and targets (use CPU to save GPU memory)
            chunk_preds.extend(preds.cpu().numpy().flatten())
            chunk_targets.extend(masks.cpu().numpy().flatten())
            
            chunk_samples += 1
            
            # Progress reporting
            if (i - start_idx) % 50 == 0:
                current_sample = start_idx + (i - start_idx)
                print(f"    Sample {current_sample}/{end_idx-1} - "
                      f"Current chunk IoU: {chunk_iou/chunk_samples:.4f}")
                
                if torch.cuda.is_available():
                    memory_allocated = torch.cuda.memory_allocated() / 1024**3
                    memory_reserved = torch.cuda.memory_reserved() / 1024**3
                    print(f"    GPU memory: {memory_allocated:.2f} GB / {memory_reserved:.2f} GB")
    
    # Calculate chunk averages
    if chunk_samples > 0:
        chunk_iou /= chunk_samples
        chunk_dice /= chunk_samples
        chunk_class_ious = [iou / chunk_samples for iou in chunk_class_ious]
        chunk_class_dice = [dice / chunk_samples for dice in chunk_class_dice]
    
    return {
        'chunk_idx': chunk_idx,
        'start_idx': start_idx,
        'end_idx': end_idx,
        'samples_processed': chunk_samples,
        'chunk_iou': chunk_iou,
        'chunk_dice': chunk_dice,
        'chunk_class_ious': chunk_class_ious,
        'chunk_class_dice': chunk_class_dice,
        'chunk_preds': chunk_preds,
        'chunk_targets': chunk_targets
    }

def update_progress(progress, chunk_results, results_dir):
    """Update overall progress with chunk results"""
    chunk_samples = chunk_results['samples_processed']
    
    if chunk_samples > 0:
        progress['current_chunk'] = chunk_results['chunk_idx'] + 1
        progress['processed_samples'] = chunk_results['end_idx']
        
        # Update cumulative metrics
        progress['total_iou'] += chunk_results['chunk_iou'] * chunk_samples
        progress['total_dice'] += chunk_results['chunk_dice'] * chunk_samples
        
        for cls in range(3):
            progress['total_class_ious'][cls] += chunk_results['chunk_class_ious'][cls] * chunk_samples
            progress['total_class_dice'][cls] += chunk_results['chunk_class_dice'][cls] * chunk_samples
        
        # Store predictions and targets
        progress['all_preds'].extend(chunk_results['chunk_preds'])
        progress['all_targets'].extend(chunk_results['chunk_targets'])
        progress['chunk_metrics'].append({
            'chunk': chunk_results['chunk_idx'],
            'iou': chunk_results['chunk_iou'],
            'dice': chunk_results['chunk_dice']
        })
    
    return progress

def save_chunk_results(chunk_results, chunk_idx, results_dir):
    """Save individual chunk results"""
    chunk_filename = f'{results_dir}/chunk_{chunk_idx:04d}.pkl'
    with open(chunk_filename, 'wb') as f:
        pickle.dump(chunk_results, f)
    print(f"💾 Chunk {chunk_idx} results saved: {chunk_filename}")

def clear_memory():
    """Clear GPU and CPU memory"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

def print_intermediate_results(progress, current_chunk):
    """Print intermediate results after each chunk"""
    total_processed = progress['processed_samples']
    if total_processed > 0:
        current_iou = progress['total_iou'] / total_processed
        current_dice = progress['total_dice'] / total_processed
        
        print(f"📊 After {current_chunk} chunks ({total_processed} samples):")
        print(f"   Current Mean IoU: {current_iou:.4f}")
        print(f"   Current Mean Dice: {current_dice:.4f}")

def save_checkpoint(progress, results_dir):
    """Save progress checkpoint"""
    checkpoint_file = f'{results_dir}/current_progress.pkl'
    with open(checkpoint_file, 'wb') as f:
        pickle.dump(progress, f)
    print(f"💾 Progress checkpoint saved")

def compile_final_results(progress, results_dir):
    """Compile final results from all chunks"""
    total_samples = progress['processed_samples']
    
    if total_samples > 0:
        mean_iou = progress['total_iou'] / total_samples
        mean_dice = progress['total_dice'] / total_samples
        class_ious = [iou / total_samples for iou in progress['total_class_ious']]
        class_dice = [dice / total_samples for dice in progress['total_class_dice']]
    else:
        mean_iou, mean_dice, class_ious, class_dice = 0.0, 0.0, [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]
    
    final_results = {
        'total_samples': total_samples,
        'mean_iou': mean_iou,
        'mean_dice': mean_dice,
        'class_ious': class_ious,
        'class_dice': class_dice,
        'all_preds': progress['all_preds'],
        'all_targets': progress['all_targets'],
        'chunk_metrics': progress['chunk_metrics']
    }
    
    # Save final results
    with open(f'{results_dir}/final_results.pkl', 'wb') as f:
        pickle.dump(final_results, f)
    
    # Clean up checkpoint
    checkpoint_file = f'{results_dir}/current_progress.pkl'
    if os.path.exists(checkpoint_file):
        os.remove(checkpoint_file)
    
    print(f"✅ Final results compiled for {total_samples} samples")
    return final_results

def plot_progress(progress, results_dir):
    """Plot metrics progress across chunks"""
    chunks = [m['chunk'] for m in progress['chunk_metrics']]
    ious = [m['iou'] for m in progress['chunk_metrics']]
    dices = [m['dice'] for m in progress['chunk_metrics']]
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(chunks, ious, 'b-', label='IoU', marker='o')
    plt.axhline(y=progress['total_iou']/progress['processed_samples'], 
                color='r', linestyle='--', label='Overall IoU')
    plt.xlabel('Chunk')
    plt.ylabel('IoU')
    plt.title('IoU Progress by Chunk')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot(chunks, dices, 'g-', label='Dice', marker='s')
    plt.axhline(y=progress['total_dice']/progress['processed_samples'], 
                color='r', linestyle='--', label='Overall Dice')
    plt.xlabel('Chunk')
    plt.ylabel('Dice')
    plt.title('Dice Progress by Chunk')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(f'{results_dir}/progress_plot.png', dpi=300, bbox_inches='tight')
    plt.show()

# ЗАПУСК ТЕСТИРОВАНИЯ
print("🧪 Loading trained model for incremental testing...")

# Определяем устройство
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU is available: {torch.cuda.get_device_name()}")
    print(f"💾 GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️  GPU not available, using CPU")

model_test = UNet(n_channels=1, n_classes=3)

try:
    checkpoint = torch.load('/kaggle/input/model/other/default/1/best_unet_liver_full.pth', map_location=device)
    model_test.load_state_dict(checkpoint)
    print("✅ Best model loaded successfully!")
except Exception as e:
    try:
        checkpoint = torch.load('final_unet_liver_full.pth', map_location=device)
        model_test.load_state_dict(checkpoint)
        print("✅ Final model loaded successfully!")
    except Exception as e:
        print(f"❌ No trained model found: {e}")
        model_test = None

if model_test is not None:
    model_test = model_test.to(device)
    model_test.eval()
    
    # Запускаем инкрементальное тестирование
    print("\n🔬 Starting incremental evaluation with memory control...")
    final_results = test_model_incremental(
        model_test, 
        val_loader_full, 
        device=device, 
        chunk_size=150,  # Обрабатываем по 150 файлов за раз
        results_dir='incremental_evaluation',
        resume_from=None  # Можно указать чанк для продолжения
    )
    
    if final_results is not None:
        # Выводим финальные результаты
        print("\n" + "="*60)
        print("🎯 FINAL MODEL EVALUATION RESULTS")
        print("="*60)
        print(f"📊 Total tested samples: {final_results['total_samples']}")
        print(f"🎯 Mean IoU: {final_results['mean_iou']:.4f}")
        print(f"🎯 Mean Dice: {final_results['mean_dice']:.4f}")
        print("\n📈 Per-class metrics:")
        class_names = ['Background', 'Liver', 'Tumor']
        for cls, (iou, dice) in enumerate(zip(final_results['class_ious'], final_results['class_dice'])):
            print(f"  {class_names[cls]:12} | IoU: {iou:.4f} | Dice: {dice:.4f}")
        
        # Benchmark quality
        avg_iou = final_results['mean_iou']
        if avg_iou > 0.7:
            print("🎉 EXCELLENT model quality!")
        elif avg_iou > 0.5:
            print("✅ GOOD model quality!")
        elif avg_iou > 0.3:
            print("⚠️  FAIR model quality - consider more training")
        else:
            print("❌ POOR model quality - needs improvement")
    
    # Очищаем память
    clear_memory()
    print("🧹 Memory cleared")
else:
    print("Please train the model first using Cell 7!")

🧪 Loading trained model for incremental testing...
✅ GPU is available: Tesla T4
💾 GPU memory: 14.7 GB
✅ Best model loaded successfully!

🔬 Starting incremental evaluation with memory control...
🧪 Starting incremental testing on 1466 samples...
📦 Chunk size: 150
💾 Results directory: incremental_evaluation

🔄 Processing chunk 1/10
📊 Samples 0 to 149
  Processing samples 0 to 149...
    Sample 0/149 - Current chunk IoU: 0.9460
    GPU memory: 0.28 GB / 2.17 GB
    Sample 50/149 - Current chunk IoU: 0.9607
    GPU memory: 0.28 GB / 2.17 GB
    Sample 100/149 - Current chunk IoU: 0.9577
    GPU memory: 0.28 GB / 2.17 GB
